In [29]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import TypedDict , Literal
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
load_dotenv()

True

In [16]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
)

In [21]:
class evaulateSchema(BaseModel):

    evaluation: Literal["approved", "need_improvement"]
    feedback: str

evaulation_structure = model.with_structured_output(evaulateSchema)

In [17]:
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "need_improvement"]
    feedback: str
    iteration: int
    max_iterations: int


In [25]:
def generate(state: TweetState):
     # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
    Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

    Rules:
    - Do NOT use question-answer format.
    - Max 280 characters.
    - Use observational humor, irony, sarcasm, or cultural references.
    - Think in meme logic, punchlines, or relatable takes.
    - Use simple, day to day english
    - Avoid cliches, platitudes, or generic humor.
    - Make it shareable and engaging.
    """)
    ]
    response = model.invoke(messages)
    return {
        "tweet": response.content}
    

In [23]:
def evaluate(state: TweetState):
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
    Evaluate the following tweet:

    Tweet: "{state['tweet']}"

    Use the criteria below to evaluate the tweet:

    1. Originality – Is this fresh, or have you seen it a hundred times before?  
    2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
    3. Punchiness – Is it short, sharp, and scroll-stopping?  
    4. Virality Potential – Would people retweet or share it?  
    5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

    Auto-reject if:
    - It's written in question-answer format (e.g., "Why did..." or "What happens when...")
    - It exceeds 280 characters
    - It reads like a traditional setup-punchline joke
    - Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

    ### Respond ONLY in structured format:
    - evaluation: "approved" or "needs_improvement"  
    - feedback: One paragraph explaining the strengths and weaknesses 
    """)
    ]
    response = evaulation_structure.invoke(messages)
    return {
        "evaluation": response.evaluation,
        "feedback": response.feedback
    }
def improve(state: TweetState):
     
    messages = [
    SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
    HumanMessage(content=f"""
    Improve the tweet based on this feedback:
    "{state['feedback']}"

    Topic: "{state['topic']}"
    Original Tweet:
    {state['tweet']}

    Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
    """)
    ]
    response = model.invoke(messages)
    return {
        "tweet": response.content,
        "iteration": state['iteration'] + 1
    } if state['iteration'] < state['max_iterations'] else END


def route_evaluation(state: TweetState):
    if state['evaluation'] == "approved":
        return END
    else:
        return 'improve'


In [27]:
graph = StateGraph(TweetState)
graph.add_node('generate', generate)
graph.add_node('evaluate', evaluate)
graph.add_node('improve', improve)

graph.add_edge(START,'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation )

workflow = graph.compile()


In [34]:
initial_state = {
    "topic":"grtrtert"
    , "iteration": 0, "max_iterations": 3, "tweet": "", "evaluation": "need_improvement", "feedback": ""
}

workflow.invoke(initial_state)

{'topic': 'grtrtert',
 'tweet': 'My autocorrect just changed "grtrtert" to "greater tert."  I\'m pretty sure my phone is judging my life choices. #autocorrectfail #existentialcrisis #sendhelp',
 'evaluation': 'approved',
 'feedback': "The tweet is mildly amusing due to the relatable nature of autocorrect errors and the humorous interpretation of the phone judging life choices. The inclusion of relevant hashtags increases virality potential. However, the originality is somewhat low, as autocorrect fails are a common theme on social media. The tweet's length is appropriate, and its format is suitable for Twitter.  While it might not be laugh-out-loud funny, it has enough appeal to garner some engagement.",
 'iteration': 0,
 'max_iterations': 3}